In [1]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. **Pre-training σε μεγάλα unlabelled datasets**:

  Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. **Fine-tuning σε μικρότερα labelled datasets**:
  
     Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του **text-classification pipeline** μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης.

Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες: **[entailment/neutral/contradiction]**.

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι **γραμματικά ορθό (acceptable) ή όχι (unacceptable)**:

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια.
Το  Yelp κατασκευάστηκε θεωρώντας τα αστέρια 1 και 2 αρνητικά και τα 3 και 4 θετικά.  Η αρνητική πολικότητα ανήκει στην κατηγορία 1 και η θετική στην κατηγορία 2. Τα reviews αυτά χωρίζονται σε αυτές τις κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.



In [2]:
from datasets import load_dataset

# insert your code here
dataset = load_dataset("yelp_polarity")

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [3]:
# insert your code here
from datasets import load_dataset, Dataset
import pandas as pd

train_labels = dataset['train']['label']
test_labels = dataset['test']['label']
categories = sorted(set(train_labels))
num_categories = len(categories)

print(f"Train Categories: {num_categories}")
print(f"Test Categories: {len(set(test_labels))}")

samples_per_category = 300 // num_categories

def balance_subset(ds, categories, max_per_cat):
    balanced = []
    for cat in categories:
        filtered = [x for x in ds if x['label'] == cat]
        balanced.extend(filtered[:max_per_cat])
    return Dataset.from_pandas(pd.DataFrame(balanced))

train_dataset = balance_subset(dataset['train'], categories, samples_per_category)
test_dataset = balance_subset(dataset['test'], categories, samples_per_category)

print(train_dataset)
print(test_dataset)


Train Categories: 2
Test Categories: 2
Dataset({
    features: ['text', 'label'],
    num_rows: 300
})
Dataset({
    features: ['text', 'label'],
    num_rows: 300
})


In [4]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα.

Η διαδικασία αυτή επιτελείται μέσω των **Tokenizers**, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, **αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer**. Επιλέγοντας τη μέθοδο **from_pretrained** λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [5]:
# insert your code here
from transformers import AutoTokenizer, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [7]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [8]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε;  Σχολιάστε και αναλύστε.

In [9]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print("Device count:", torch.cuda.device_count())
    print("Current device index:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
    print("Device capability:", torch.cuda.get_device_capability(torch.cuda.current_device()))
else:
    print("CUDA is not available.")

CUDA is available!
Device count: 1
Current device index: 0
Device name: NVIDIA GeForce RTX 3060
Device capability: (8, 6)


In [10]:
!pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [11]:
import transformers
print(transformers.__version__)
from transformers import TrainingArguments
import inspect
sig = inspect.signature(TrainingArguments)
sig_str = str(sig)

supported_args = set(sig.parameters.keys())

if 'evaluation_strategy' in sig_str:
    print('found')
else:
    print('not found')


4.51.3
not found


In [12]:
! pip install -U accelerate
! pip install -U transformers
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, default_data_collator
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
sig = inspect.signature(TrainingArguments)
print(sig)

# insert your code here

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
def get_loader(ds, batch_size, shuffle=False):
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                      collate_fn=default_data_collator)

def train_one_epoch(model, loader, optimizer, scheduler):
    model.train()
    for batch in loader:
        batch = {k: v.to(device) for k,v in batch.items()}
        loss = model(**batch).loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    torch.cuda.empty_cache()

def eval_acc(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for batch in loader:
            labels = batch["labels"].to(device)
            inputs = {k:v.to(device) for k,v in batch.items() if k!="labels"}
            logits = model(**inputs).logits
            preds = logits.argmax(dim=-1)
            correct += (preds==labels).sum().item()
            total   += labels.size(0)
    return correct/total

lrs         = [5e-6, 5e-5, 2e-5, 3e-3]
batch_sizes = [16, 32]
epochs_list = [3, 5, 7, 9, 18]
results     = []
for lr in lrs:
    for bs in batch_sizes:
        for num_epochs in epochs_list:
            print(f"\n→ lr={lr}, batch_size={bs}, epochs={num_epochs}")
            # reload fresh model for each run
            model = DistilBertForSequenceClassification.from_pretrained(
                "distilbert-base-uncased", num_labels=2
            ).to(device)

            train_loader = get_loader(train_dataset, bs, shuffle=True)
            eval_loader  = get_loader(test_dataset,  bs, shuffle=False)

            optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
            total_steps = len(train_loader) * num_epochs
            scheduler   = torch.optim.lr_scheduler.LinearLR(
                optimizer, start_factor=1.0, end_factor=0.0, total_iters=total_steps
            )

            # run training
            for _ in range(num_epochs):
                train_one_epoch(model, train_loader, optimizer, scheduler)

            # evaluate
            acc = eval_acc(model, eval_loader)
            print(f"  → accuracy: {acc:.4f}")
            results.append((lr, bs, num_epochs, acc))

# Results table
df_results = pd.DataFrame(results, columns=["Learning Rate", "Batch Size", "Epochs", "Accuracy"])
print(df_results)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

  → accuracy: 0.5000

→ lr=0.003, batch_size=32, epochs=9


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  → accuracy: 0.5000

→ lr=0.003, batch_size=32, epochs=18


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  → accuracy: 0.5000
    Learning Rate  Batch Size  Epochs  Accuracy
0        0.000005          16       3  0.620000
1        0.000005          16       5  0.790000
2        0.000005          16       7  0.800000
3        0.000005          16       9  0.856667
4        0.000005          16      18  0.913333
5        0.000005          32       3  0.773333
6        0.000005          32       5  0.706667
7        0.000005          32       7  0.876667
8        0.000005          32       9  0.796667
9        0.000005          32      18  0.903333
10       0.000050          16       3  0.913333
11       0.000050          16       5  0.900000
12       0.000050          16       7  0.930000
13       0.000050          16       9  0.886667
14       0.000050          16      18  0.916667
15       0.000050          32       3  0.903333
16       0.000050          32       5  0.906667
17       0.000050          32       7  0.900000
18       0.000050          32       9  0.906667
19       0.000050  

In [13]:
print("| Learning Rate | Batch Size | Epochs | Accuracy |")
print("|---------------|------------|--------|----------|")
for lr, batch_size, epoch, accuracy in results:
    print(f"| {lr}           | {batch_size}          | {epoch}      | {accuracy:.4f}   |")

| Learning Rate | Batch Size | Epochs | Accuracy |
|---------------|------------|--------|----------|
| 5e-06           | 16          | 3      | 0.6200   |
| 5e-06           | 16          | 5      | 0.7900   |
| 5e-06           | 16          | 7      | 0.8000   |
| 5e-06           | 16          | 9      | 0.8567   |
| 5e-06           | 16          | 18      | 0.9133   |
| 5e-06           | 32          | 3      | 0.7733   |
| 5e-06           | 32          | 5      | 0.7067   |
| 5e-06           | 32          | 7      | 0.8767   |
| 5e-06           | 32          | 9      | 0.7967   |
| 5e-06           | 32          | 18      | 0.9033   |
| 5e-05           | 16          | 3      | 0.9133   |
| 5e-05           | 16          | 5      | 0.9000   |
| 5e-05           | 16          | 7      | 0.9300   |
| 5e-05           | 16          | 9      | 0.8867   |
| 5e-05           | 16          | 18      | 0.9167   |
| 5e-05           | 32          | 3      | 0.9033   |
| 5e-05           | 32         

# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή λογική (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [14]:
# # insert your code here (load dataset)
from datasets import load_dataset
piqa = load_dataset("piqa")

piqa_train_subset = piqa["train"].shuffle(seed=42).select(range(100))
piqa_test_subset  = piqa["test"].shuffle(seed=42).select(range(100))

print(f"Train subset size: {len(piqa_train_subset)}")
print(f"Test  subset size: {len(piqa_test_subset)}")
print("\nexample from the test set:\n", piqa_test_subset[0])

Train subset size: 100
Test  subset size: 100

example from the test set:
 {'goal': 'how do you paint with light?', 'sol1': 'You use a camera with a long shutter exposure, and wave a light source around.', 'sol2': 'The best way, is to set up an easel and take a light source and hold it close to your canvas.', 'label': -1}


Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [15]:
# insert your code here (models)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
model_names = ["bert-base-uncased", "xlnet-base-cased", "roberta-base", "t5-small", "distilbert-base-uncased"]
tokenizers = []
models = []
for model_name in model_names:
    tokenizers.append(AutoTokenizer.from_pretrained(model_name))
    models.append(AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

In [16]:
# insert your code here (function for ending prediction)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#cpu = torch.device("cpu")
torch.cuda.empty_cache()
def end_to_end_prediction(tokenizer, model, dataset):
    model.eval()
    model.to(device)
    inputs1 = tokenizer(dataset["goal"], dataset["sol1"], padding=True, truncation=True, return_tensors="pt").to(device)
    inputs2 = tokenizer(dataset["goal"], dataset["sol2"], padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)
    return torch.argmax(outputs1.logits, dim=1).cpu().numpy().tolist()

predictions = []
for i in range(len(models)):
    predictions += end_to_end_prediction(tokenizers[i], models[i], piqa_test_subset)
    print(f"Model: {model_names[i]}")
    print(f"Predictions: {predictions[i]}")
    print(f"Labels: {piqa_test_subset['label']}")
    print("")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Model: bert-base-uncased
Predictions: 1
Labels: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

Model: xlnet-base-cased
Predictions: 0
Labels: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]



Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Model: roberta-base
Predictions: 0
Labels: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

Model: t5-small
Predictions: 1
Labels: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

Model: distilbert-base-uncased
Predictions: 0
Labels: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

## B2. Truthful QA

### Sentence Transformers

Οι **sentence transformers** χρησιμοποιούνται για να δημιουργήσουν **embeddings προτάσεων**, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [18]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

np.float32(0.40488476)

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [19]:
# insert your code here (load dataset)
from datasets import load_dataset
import random
from datasets import Dataset
dataset = load_dataset("truthful_qa",'generation', trust_remote_code=True)
print(dataset)
filtered_dataset = []
for sample in dataset['validation']:
    if len(sample['correct_answers']) >= 2:
        filtered_dataset.append(sample)
    if len(filtered_dataset) >= 100:
        break
filtered_dataset = Dataset.from_list(filtered_dataset)
print(filtered_dataset)

DatasetDict({
    validation: Dataset({
        features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source'],
        num_rows: 817
    })
})
Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source'],
    num_rows: 100
})


In [20]:
# insert your code here (load models for semantic similarity and QA)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer

model_names = ['bert-base-uncased', 'xlnet-base-cased', 'roberta-base', 't5-small', 'distilbert-base-uncased']

sentence_model_names = ['all-MiniLM-L6-v2',
    'all-mpnet-base-v2',
    'all-MiniLM-L12-v2',
    'paraphrase-multilingual-MiniLM-L12-v2',
    'bert-base-nli-mean-tokens',
    'multi-qa-MiniLM-L6-cos-v1'
    ]


tokenizers = [ AutoTokenizer.from_pretrained(model_name) for model_name in model_names ]
models = [ AutoModelForSequenceClassification.from_pretrained(model_name) for model_name in model_names ]

sentence_models = [ SentenceTransformer('sentence-transformers/' + sentence_model_name) for sentence_model_name in sentence_model_names ]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

In [21]:
# insert your code here (function for optimal correct answers & semantic similarity)
import torch

for m in models:
    m.to(device)

def predict_best_answer(question, answers, model, tokenizer, sentence_model):
    model.eval()
    # batch‐tokenize all (question + answer) pairs
    max_len = getattr(tokenizer, "model_max_length", None)
    if (not isinstance(max_len, int)) or max_len > 512:
        max_len = 512

    encodings = tokenizer(
        [question + " " + a for a in answers],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_len,
    ).to(device)

    # forward pass
    with torch.no_grad():
        logits = model(**encodings).logits  # (batch_size, num_labels)

    # score each candidate by logit difference (class 1 – class 0) or logit itself if single‐label
    if logits.size(-1) > 1:
        scores = logits[:, 1] - logits[:, 0]
    else:
        scores = logits.squeeze(-1)

    best_idx = scores.argmax().item()
    return answers[best_idx]

def evaluate_accuracy(filtered_dataset, model, tokenizer, sentence_model):
    model.eval()
    correct = 0
    total = len(filtered_dataset)

    for sample in filtered_dataset:
        question    = sample["question"]
        answers     = sample["correct_answers"][:2] + [sample["incorrect_answers"][0]] + [sample["best_answer"]]
        best_answer = predict_best_answer(question, answers, model, tokenizer, sentence_model)
        if best_answer == sample["best_answer"]:
            correct += 1

    return correct / total if total > 0 else 0.0
    
for i in range(len(models)):
    print(f"Model: {model_names[i]}")
    for j in range(len(sentence_models)):
        acc = evaluate_accuracy(filtered_dataset, models[i], tokenizers[i], sentence_models[j])
        print(f"Sentence Model: {sentence_model_names[j]}")
        print(f"Accuracy: {acc:.4f}\n")


Model: bert-base-uncased
Sentence Model: all-MiniLM-L6-v2
Accuracy: 0.3100

Sentence Model: all-mpnet-base-v2
Accuracy: 0.3100

Sentence Model: all-MiniLM-L12-v2
Accuracy: 0.3100

Sentence Model: paraphrase-multilingual-MiniLM-L12-v2
Accuracy: 0.3100

Sentence Model: bert-base-nli-mean-tokens
Accuracy: 0.3100

Sentence Model: multi-qa-MiniLM-L6-cos-v1
Accuracy: 0.3100

Model: xlnet-base-cased
Sentence Model: all-MiniLM-L6-v2
Accuracy: 0.3200

Sentence Model: all-mpnet-base-v2
Accuracy: 0.3200

Sentence Model: all-MiniLM-L12-v2
Accuracy: 0.3200

Sentence Model: paraphrase-multilingual-MiniLM-L12-v2
Accuracy: 0.3200

Sentence Model: bert-base-nli-mean-tokens
Accuracy: 0.3200

Sentence Model: multi-qa-MiniLM-L6-cos-v1
Accuracy: 0.3200

Model: roberta-base
Sentence Model: all-MiniLM-L6-v2
Accuracy: 0.3400

Sentence Model: all-mpnet-base-v2
Accuracy: 0.3400

Sentence Model: all-MiniLM-L12-v2
Accuracy: 0.3400

Sentence Model: paraphrase-multilingual-MiniLM-L12-v2
Accuracy: 0.3400

Sentence M

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [22]:
# insert your code here (load dataset)
from datasets import load_dataset
dataset = load_dataset("winogrande", "winogrande_xl")
train_dataset = dataset['train'].shuffle(seed=42)
train_dataset = train_dataset.flatten_indices()
train_dataset = Dataset.from_dict(train_dataset[:100])

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [23]:
# insert your code here (load models)
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_names = ["bert-base-uncased", "roberta-base", "distilbert-base-uncased"]

tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in model_names]
models = [AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) for model_name in model_names]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [24]:
# insert your code here (create pipelines)
from transformers import pipeline

pipelines = [pipeline('text-classification', model=model, tokenizer=tokenizer) for model, tokenizer in zip(models, tokenizers)]

Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


In [25]:
# insert your code here (function for predicting best fill)
def predict_best_fill(sentence, option1, option2, pipeline):
    sentence1 = sentence.replace("_", option1)
    result1 = pipeline(sentence1)
    sentence2 = sentence.replace("_", option2)
    result2 = pipeline(sentence2)
    return 0 if result1[0]['score'] > result2[0]['score'] else 1

def evaluate_accuracy(train_dataset, pipelines):
    
    accuracy = []
    for pipeline in pipelines:
        correct_predictions = 0
        total_predictions = 0
        for sample in train_dataset:
            question = sample['sentence']
            option1 = sample['option1']
            option2 = sample['option2']
            label = int(sample['answer'])-1

            best_fill = predict_best_fill(question, option1, option2, pipeline)

            if best_fill == label:
                correct_predictions += 1

            total_predictions += 1

        accuracy.append(correct_predictions / total_predictions)
    return accuracy

print(evaluate_accuracy(train_dataset, pipelines))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[0.44, 0.52, 0.54]
